In [ ]:
%matplotlib notebook

import subprocess
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from time import time

In [ ]:
def plot(fig, ax_nagg, ax_kfe, ax_dfe, ax_cpu_coll, ax_cpu_it,
         Nagg, Time, CPU, it, kfe, dfe, r, cpu_per_coll, cpu_per_it, maxcpu_coll):
    
    ncoll = len(Nagg)
    
    if ax_nagg.lines:
        ax_nagg.lines[0].set_xdata(Time)
        ax_nagg.lines[0].set_ydata(Nagg)
    else:
        ax_nagg.semilogx(Time, Nagg)
        ax_nagg.set_title('Nagg')
    ax_nagg.set_xlim(max(1e-10,min(Time))*0.8, max(Time)*1.2)
    ax_nagg.set_ylim(min(Nagg)*0.8, max(Nagg)*1.2)

    if ax_kfe.lines:
        ax_kfe.lines[0].set_xdata(Time)
        ax_kfe.lines[0].set_ydata(kfe)
        ax_dfe.lines[0].set_xdata(Time)
        ax_dfe.lines[0].set_ydata(dfe)
    else:
        l1 = ax_kfe.semilogx(Time, kfe, label="kfe")
        l2 = ax_dfe.semilogx(Time, dfe, "r", label="dfe")
        ax_kfe.set_title('dfe and kfe')
        ax_kfe.set(ylabel='kfe')
        ax_dfe.set(ylabel='dfe')
        lns = l1+l2
        labs = [l.get_label() for l in lns]
        ax_kfe.legend(lns, labs, loc=0)
    ax_kfe.set_xlim(max(1e-10,min(Time))*0.8, max(Time)*1.2)
    ax_kfe.set_ylim(min(kfe)*0.8, max(kfe)*1.2)
    ax_dfe.set_ylim(min(dfe)*0.8, max(dfe)*1.2)

    if ax_cpu_coll.lines:
        ax_cpu_coll.lines[0].set_xdata(Time)
        ax_cpu_coll.lines[1].set_xdata(Time)
        ax_cpu_coll.lines[0].set_ydata(cpu_per_coll)
        ax_cpu_coll.lines[1].set_ydata(maxcpu_coll)
    else:
        ax_cpu_coll.loglog(Time, cpu_per_coll)
        ax_cpu_coll.loglog(Time, maxcpu_coll)
        ax_cpu_coll.set_title('cpu time per collision')

    ax_cpu_coll.set_xlim(max(1e-10,min(Time))*0.8, max(Time)*1.2)
    ax_cpu_coll.set_ylim(max(1e-5,min(cpu_per_coll))*0.8, max(cpu_per_coll)*1.2)
    
    if ax_cpu_it.lines:
        ax_cpu_it.lines[0].set_xdata(Time)
        ax_cpu_it.lines[0].set_ydata(cpu_per_it)
    else:
        ax_cpu_it.loglog(Time, cpu_per_it)
        ax_cpu_it.set_title('cpu time per it')
    ax_cpu_it.set_xlim(max(1e-10,min(Time))*0.8, max(Time)*1.2)
    ax_cpu_it.set_ylim(max(1e-6,min(cpu_per_it))*0.8, max(cpu_per_it)*1.2)
    
    fig.canvas.draw()



In [ ]:
def run_DLA_interactive_plot():
    fig, ((ax_nagg, ax_kfe), (ax_cpu_coll, ax_cpu_it)) = plt.subplots(2, 2,figsize=(8,6)) 
    ax_dfe = ax_kfe.twinx()

    oldcpu = 0.
    maxcpu_coll= 0.
    ncoll=0
    Nagg = []
    Time = []
    CPU = []
    it = []
    kfe = []
    dfe = []
    r = []
    cpu_per_coll = []
    cpu_per_it = []
    maxcpu_coll = []
    t0 = time()-1
    with subprocess.Popen("../bin/release/DLCA params.txt".split(),
                         stdout=subprocess.PIPE) as DLCA:
        for line in iter(DLCA.stdout.readline, b''):
            line = line.decode()
            if "NAgg" not in line:
                continue

            ncoll += 1
            Nagg.append(int(line.split("=")[1]
                                .split("Time")[0]))
            Time.append(float(line.split("=")[2]
                                  .split("s")[0]))
            CPU.append(float(line.split("=")[3]
                                 .split("s")[0]))
            it.append(int(line.split("after")[1]
                              .split("it")[0]))

            kfe.append(float(line.split("---")[1]
                                 .split("*")[0]))
            dfe.append(float(line.split("x^")[1]
                                 .split("---")[0]))

            r.append(float(line.split("r=")[1]))

            cpu_per_coll.append(CPU[-1] - oldcpu)
            cpu_per_it.append(cpu_per_coll[-1] / (it[-1] + 1))
            oldcpu = CPU[-1]
            maxcpu_coll.append(max(cpu_per_coll))
            t1 = time()
            if t1 - t0 > 0.2:
                plot(fig, ax_nagg, ax_kfe, ax_dfe, ax_cpu_coll, ax_cpu_it,
                     Nagg, Time, CPU, it, kfe, dfe, r, cpu_per_coll, cpu_per_it, maxcpu_coll)
                t0 = t1
                
    return Nagg, Time, CPU, it, kfe, dfe, r, cpu_per_coll, cpu_per_it, maxcpu_coll



In [ ]:
from pathlib import Path

def get_output_path():
    for line in open("params.txt"):
        value = line.split()[0]
    return Path(value)

def check_output_path(force=False):
    if (list(get_output_path().glob("*.h5")) +
        list(get_output_path().glob("*.xdmf"))):
        
        if not force:
            raise OSError("Output folder not empty")
            
        print("Output folder not empty")
            

In [ ]:
check_output_path(force=False)
data = run_DLA_interactive_plot()
print(data[1][-1])

In [ ]:
!../build-Generateur_DLCA/release/DLCA params.txt